In [100]:
import pandas as pd
import os

In [101]:
ORIGINAL_DATA_DIR = "data/original/"

ORIGINAL_GG = "data/original/gg_concatenado.csv"
ORIGINAL_OSCAR = "data/original/oscar.csv"
ORIGINAL_SAG = "data/original/sag.csv"
ORIGINAL_BAFTA = "data/original/bafta.csv"
ORIGINAL_CRITICS = "data/original/critics.csv"

TRANS_DATA_DIR = "data/transformed/"

In [102]:
df_gg = pd.read_csv(ORIGINAL_GG)
df_oscars = pd.read_csv(ORIGINAL_OSCAR)
df_sag = pd.read_csv(ORIGINAL_SAG)
df_bafta = pd.read_csv(ORIGINAL_BAFTA)
df_critics = pd.read_csv(ORIGINAL_CRITICS)

In [103]:
df_gg_2014_2024 = df_gg[(df_gg['ano'] >= 2014) & (df_gg['ano'] <= 2024)]

df_oscars_2014_2024 = df_oscars[(df_oscars['ano'] >= 2014) & (df_oscars['ano'] <= 2024)]

df_sag_2014_2024 = df_sag[(df_sag['ano'] >= 2014) & (df_sag['ano'] <= 2024)]

df_bafta_2014_2024 = df_bafta[(df_bafta['ano'] >= 2014) & (df_bafta['ano'] <= 2024)]

df_critics_2014_2024 = df_critics[(df_critics['ano'] >= 2014) & (df_critics['ano'] <= 2024)]

In [104]:
#copias explicitas para evitar o SettingWithCopyWarning
df_gg_2014_2024 = df_gg_2014_2024.copy()
df_oscars_2014_2024 = df_oscars_2014_2024.copy()
df_sag_2014_2024 = df_sag_2014_2024.copy()
df_bafta_2014_2024 = df_bafta_2014_2024.copy()
df_critics_2014_2024 = df_critics_2014_2024.copy()

#criando colunas de indicacao e rennomeando ganhou
df_gg_2014_2024['indicada_gg'] = True
df_gg_2014_2024.rename(columns={'ganhou': 'ganhou_gg'}, inplace=True)


df_oscars_2014_2024['indicada_os'] = True
df_oscars_2014_2024.rename(columns={'ganhou': 'ganhou_os'}, inplace=True)

df_sag_2014_2024['indicada_sag'] = True
df_sag_2014_2024.rename(columns={'ganhou': 'ganhou_sag'}, inplace=True)

df_bafta_2014_2024['indicada_bafta'] = True
df_bafta_2014_2024.rename(columns={'ganhou': 'ganhou_bafta'}, inplace=True)

df_critics_2014_2024['indicada_critics'] = True
df_critics_2014_2024.rename(columns={'ganhou': 'ganhou_critics'}, inplace=True)

In [105]:
df_bafta_2014_2024.head()

,ano,nome,personagem,filme,ganhou_bafta,indicada_bafta
19,2014,Julianne Moore,Dr. Alice Howland,Still Alice,True,True
20,2014,Amy Adams,Margaret Keane,Big Eyes,False,True
21,2014,Felicity Jones,Jane Wilde Hawking,The Theory of Everything,False,True
22,2014,Rosamund Pike,Amy Elliott-Dunne,Gone Girl,False,True
23,2014,Reese Witherspoon,Cheryl Strayed,Wild,False,True


In [106]:
print(df_critics_2014_2024.head())

     ano              nome          personagem                         filme  \
78  2014    Julianne Moore   Dr. Alice Howland              Still Alice \r\n   
79  2014  Jennifer Aniston      Claire Bennett                      Cake\r\n   
80  2014  Marion Cotillard          Sandra Bya       Two Days, One Night\r\n   
81  2014    Felicity Jones  Jane Wilde Hawking  The Theory of Everything\r\n   
82  2014     Rosamund Pike   Amy Elliott-Dunne                 Gone Girl\r\n   

    ganhou_critics  indicada_critics  
78            True              True  
79           False              True  
80           False              True  
81           False              True  
82           False              True  


In [107]:
ganhou_gg_count = df_gg_2014_2024['ganhou_gg'].sum()
ganhou_oscar_count = df_oscars_2014_2024['ganhou_os'].sum()
ganhou_sag_count = df_sag_2014_2024['ganhou_sag'].sum()
ganhou_bafta_count = df_bafta_2014_2024['ganhou_bafta'].sum()
ganhou_critics_count = df_critics_2014_2024['ganhou_critics'].sum()

print(f"Quantidade de 'ganhou_gg' True: {ganhou_gg_count}")
print(f"Quantidade de 'ganhou_os' True: {ganhou_oscar_count}")
print(f"Quantidade de 'ganhou_sag' True: {ganhou_sag_count}")
print(f"Quantidade de 'ganhou_bafta' True: {ganhou_bafta_count}")
print(f"Quantidade de 'ganhou_critics' True: {ganhou_critics_count}")

Quantidade de 'ganhou_gg' True: 22
Quantidade de 'ganhou_os' True: 10
Quantidade de 'ganhou_sag' True: 11
Quantidade de 'ganhou_bafta' True: 11
Quantidade de 'ganhou_critics' True: 12


In [108]:
df_oscars_2014_2024.shape

(55, 6)

In [109]:
dfs = [
    (df_gg_2014_2024, 'indicada_gg', 'ganhou_gg'),
    (df_oscars_2014_2024, 'indicada_os', 'ganhou_os'),
    (df_sag_2014_2024, 'indicada_sag', 'ganhou_sag'),
    (df_bafta_2014_2024, 'indicada_bafta', 'ganhou_bafta'),
    (df_critics_2014_2024, 'indicada_critics', 'ganhou_critics')
] 

Pivotar a tabela

In [110]:
def processar_df(df, indicada_col, ganhou_col):
    df_pivot = pd.pivot_table(
        df,
        index='nome',
        columns='ano',
        values=indicada_col,  
        aggfunc='any',  
        fill_value=False
    )

    df_pivot.columns = [f'participou_{ano}' for ano in df_pivot.columns]

    indicacoes = df.groupby('nome')[indicada_col].max()
    vitorias = df.groupby('nome')[ganhou_col].max()

    df_final = pd.concat([df_pivot, indicacoes, vitorias], axis=1)
    return df_final

dfs_final = [processar_df(df, indicada_col, ganhou_col) for df, indicada_col, ganhou_col in dfs]

In [111]:
df_oscars_2014_2024.shape

(55, 6)

In [112]:
df_oscars_2014_2024.head()

,ano,nome,personagem,filme,ganhou_os,indicada_os
429,2014,Julianne Moore,Alice Howland,Still Alice,True,True
430,2014,Marion Cotillard,Sandra Bya,"Two Days, One Night",False,True
431,2014,Felicity Jones,Jane Wilde Hawking,The Theory of Everything,False,True
432,2014,Rosamund Pike,Amy Elliott-Dunne,Gone Girl,False,True
433,2014,Reese Witherspoon,Cheryl Strayed,Wild,False,True


In [113]:
unique_names_oscars = df_oscars_2014_2024['nome'].nunique()

print(f"Nomes sem repetição: {unique_names_oscars}")

Nomes sem repetição: 46


In [114]:
dfs_final[1].shape

(46, 13)

In [115]:
#procurando as coitadas que o processo comeu
merged_df = df_oscars_2014_2024.merge(dfs_final[1], on='nome', how='outer', indicator=True)
diff_df = merged_df[merged_df['_merge'] != 'both']

diff_df

,ano,nome,personagem,filme,ganhou_os_x,indicada_os_x,participou_2014,participou_2015,participou_2016,participou_2017,participou_2018,participou_2019,participou_2020,participou_2021,participou_2022,participou_2023,participou_2024,indicada_os_y,ganhou_os_y,_merge


In [116]:
ganhou_gg_count = dfs_final[0]['ganhou_gg'].sum()
ganhou_oscar_count = dfs_final[1]['ganhou_os'].sum()
ganhou_sag_count = dfs_final[2]['ganhou_sag'].sum()
ganhou_bafta_count = dfs_final[3]['ganhou_bafta'].sum()
ganhou_critics_count = dfs_final[4]['ganhou_critics'].sum()

print(f"Quantidade de 'ganhou_gg' 1: {ganhou_gg_count}")
print(f"Quantidade de 'ganhou_os' 1: {ganhou_oscar_count}")
print(f"Quantidade de 'ganhou_sag' 1: {ganhou_sag_count}")
print(f"Quantidade de 'ganhou_bafta' 1: {ganhou_bafta_count}")
print(f"Quantidade de 'ganhou_critics' 1: {ganhou_critics_count}")

Quantidade de 'ganhou_gg' 1: 21
Quantidade de 'ganhou_os' 1: 8
Quantidade de 'ganhou_sag' 1: 11
Quantidade de 'ganhou_bafta' 1: 9
Quantidade de 'ganhou_critics' 1: 12


In [117]:
dfs_final[0].to_csv('data/transformed/gg_piv.csv')
print('GG:', dfs_final[0].columns[-1])

dfs_final[1].to_csv('data/transformed/oscar_piv.csv')
print('Oscar:', dfs_final[1].columns[-1])

dfs_final[2].to_csv('data/transformed/sag_piv.csv')
print('SAG:', dfs_final[2].columns[-1])

dfs_final[3].to_csv('data/transformed/bafta_piv.csv')
print('BAFTA:', dfs_final[3].columns[-1])

dfs_final[4].to_csv('data/transformed/critics_piv.csv')
print('Critics:', dfs_final[4].columns[-1])

GG: ganhou_gg
Oscar: ganhou_os
SAG: ganhou_sag
BAFTA: ganhou_bafta
Critics: ganhou_critics
